In [1]:
import random

# for data, model, training
import pandas as pd
import numpy as np
import tensorflow as tf
import librosa

import matplotlib.pyplot as plt
import seaborn as sns

import basics

# Set the seed value for experiment reproducibility.
seed = 42
random.seed(42)
tf.random.set_seed(seed)
np.random.seed(seed)

In [2]:
filenames = basics.get_and_shuffle_filenames("./recordings")

print(filenames[:5])

['./recordings/2_jackson_13.wav', './recordings/6_george_34.wav', './recordings/7_george_5.wav', './recordings/1_yweweler_21.wav', './recordings/2_george_42.wav']


In [3]:
X_unfiltered = [(file_path, basics.decode_audio(file_path)) for file_path in filenames]

# to remove outliers 
max_length = basics.get_max_length(X_unfiltered)
print(max_length)

5632


In [4]:
X_full = [] # padded X values 0-7
y_full = []

numbers = [0] * 8

for file_path, audio in X_unfiltered:
    x_val = audio
    y_val = basics.get_label(file_path)
    signal_length = audio.shape[0]
    
    if y_val > 7:
        continue
    if signal_length > max_length:
        numbers[y_val] += 1
        continue
        
    x_val = np.pad(
        x_val, (0, max_length - signal_length), 
        'constant', constant_values=(0, 0))

    x_spect = basics.spect(x_val, max_length)
    x_spect = x_spect.flatten()

    X_full.append(x_spect)
    y_full.append(y_val)

X_full = np.array(X_full)
y_full = np.array(y_full)

print(X_full.shape)

num_samples, sample_w = X_full.shape
print(num_samples)
print(sample_w)

print(y_full[:10])

(2312, 2816)
2312
2816
[2 6 7 1 2 6 6 4 3 2]


In [5]:
# dropped outliers
df = pd.DataFrame.from_dict({"quantities": numbers})
print(df)
print(sum(numbers))

   quantities
0          12
1           9
2           7
3           9
4           3
5           8
6          29
7          11
88


In [6]:
# normalize data
X_full = basics.normalize_arr(X_full)

# partition into 80:10:10
partitions = basics.split_full(X_full, y_full)

X_train, y_train = partitions[0]
X_val, y_val = partitions[1]
X_test, y_test = partitions[2]

print('Training set size', len(X_train))
print('Validation set size', len(X_val))
print('Test set size', len(X_test))

Training set size 1848
Validation set size 231
Test set size 233


In [8]:
# INPUTS ARE NORMALIZED

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Reshape((128, 22, 1), input_shape=(sample_w,)))
model.add(tf.keras.layers.Conv2D(64, (4, 4), activation='relu', input_shape=(128, 22, 1)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.AveragePooling2D(2,2))
model.add(tf.keras.layers.Conv2D(16, (4, 4), activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.AveragePooling2D(2,2))
 
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dropout(0.15))    
model.add(tf.keras.layers.Dense(70, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='softmax'))

model.build()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 128, 22, 1)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 125, 19, 64)       1088      
                                                                 
 dropout_3 (Dropout)         (None, 125, 19, 64)       0         
                                                                 
 average_pooling2d_2 (Averag  (None, 62, 9, 64)        0         
 ePooling2D)                                                     
                                                                 
 conv2d_3 (Conv2D)           (None, 59, 6, 16)         16400     
                                                                 
 dropout_4 (Dropout)         (None, 59, 6, 16)         0         
                                                      

In [9]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer="adam",
             metrics=["accuracy"])

In [10]:
history = model.fit(X_train, y_train, epochs=30,
                   validation_data=(X_val, y_val))

Epoch 1/30
58/58 [==============================] - 5s 84ms/step - loss: 1.9315 - accuracy: 0.2587 - val_loss: 1.6350 - val_accuracy: 0.4416
Epoch 2/30
58/58 [==============================] - 4s 76ms/step - loss: 1.2088 - accuracy: 0.5758 - val_loss: 0.8931 - val_accuracy: 0.7186
Epoch 3/30
58/58 [==============================] - 4s 74ms/step - loss: 0.7928 - accuracy: 0.7143 - val_loss: 0.7109 - val_accuracy: 0.7835
Epoch 4/30
58/58 [==============================] - 4s 76ms/step - loss: 0.5717 - accuracy: 0.8025 - val_loss: 0.4663 - val_accuracy: 0.8918
Epoch 5/30
58/58 [==============================] - 5s 80ms/step - loss: 0.4119 - accuracy: 0.8631 - val_loss: 0.4300 - val_accuracy: 0.8745
Epoch 6/30
58/58 [==============================] - 4s 77ms/step - loss: 0.3271 - accuracy: 0.8912 - val_loss: 0.4189 - val_accuracy: 0.8831
Epoch 7/30
58/58 [==============================] - 4s 72ms/step - loss: 0.2622 - accuracy: 0.9123 - val_loss: 0.3094 - val_accuracy: 0.8961
Epoch 8/30
58

In [11]:
model.evaluate(X_test, y_test)

8/8 [==============================] - 0s 12ms/step - loss: 0.1567 - accuracy: 0.9528


[0.15665334463119507, 0.9527897238731384]

In [12]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 15
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = len(X_train)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(loss="sparse_categorical_crossentropy",
             optimizer="adam",
             metrics=["accuracy"])

model_for_pruning.summary()

/Users/jasminewu/Documents/6115/II/Spoken_Digits/scripts/venv/lib/python3.7/site-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshape  (None, 128, 22, 1)       1         
 _1 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_conv2d_  (None, 125, 19, 64)      2114      
 2 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_dropout  (None, 125, 19, 64)      1         
 _3 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_average  (None, 62, 9, 64)        1         
 _pooling2d_2 (PruneLowMagni                                     
 tude)                                                           
                                                      

/Users/jasminewu/Documents/6115/II/Spoken_Digits/scripts/venv/lib/python3.7/site-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/Users/jasminewu/Documents/6115/II/Spoken_Digits/scripts/venv/lib/python3.7/site-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)


 _pooling2d_3 (PruneLowMagni                                     
 tude)                                                           
                                                                 
 prune_low_magnitude_flatten  (None, 1392)             1         
 _1 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_dropout  (None, 1392)             1         
 _5 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_dense_2  (None, 70)               194952    
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense_3  (None, 8)                1130      
  (PruneLowMagnitude)                                            
                                                                 
Total para

In [15]:
import tempfile

logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(X_train, y_train,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/15
13/13 [==============================] - 8s 320ms/step - loss: 0.0308 - accuracy: 0.9910 - val_loss: 0.0093 - val_accuracy: 1.0000
Epoch 2/15
13/13 [==============================] - 4s 315ms/step - loss: 0.0413 - accuracy: 0.9892 - val_loss: 0.0147 - val_accuracy: 1.0000
Epoch 3/15
13/13 [==============================] - 4s 308ms/step - loss: 0.0316 - accuracy: 0.9886 - val_loss: 0.0093 - val_accuracy: 1.0000
Epoch 4/15
13/13 [==============================] - 4s 302ms/step - loss: 0.0245 - accuracy: 0.9922 - val_loss: 0.0127 - val_accuracy: 0.9946
Epoch 5/15
13/13 [==============================] - 4s 304ms/step - loss: 0.0287 - accuracy: 0.9910 - val_loss: 0.0164 - val_accuracy: 0.9946
Epoch 6/15
13/13 [==============================] - 4s 311ms/step - loss: 0.0226 - accuracy: 0.9910 - val_loss: 0.0185 - val_accuracy: 0.9946
Epoch 7/15
13/13 [==============================] - 4s 307ms/step - loss: 0.0201 - accuracy: 0.9946 - val_loss: 0.0071 - val_accuracy: 1.0000
Epoch 

In [16]:
model_for_pruning.evaluate(X_test, y_test)

8/8 [==============================] - 0s 14ms/step - loss: 0.1560 - accuracy: 0.9571


[0.1560189127922058, 0.9570815563201904]

In [22]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: /var/folders/vs/b_y62_cj4859tj4h1ch0glg80000gn/T/tmpgla1a4f4.h5
